In [9]:
pip install google-adk google-genai litellm python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [10]:
# Notebook-friendly setup for ADK + LiteLLM (Gemini via LiteLLM Proxy)

# ---- Imports ----
from dotenv import load_dotenv
import os

from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm

import litellm

# ---- Load environment variables ----
load_dotenv()

# ---- Enable LiteLLM proxy mode ----
litellm.use_litellm_proxy = True

# ---- (Optional) Quick sanity checks ----
assert os.getenv("LITELLM_PROXY_API_BASE"), "Missing LITELLM_PROXY_API_BASE"
assert os.getenv("LITELLM_PROXY_GEMINI_API_KEY"), "Missing LITELLM_PROXY_GEMINI_API_KEY"

# ---- Configure LiteLLM-backed Gemini model ----
lite_llm_model = LiteLlm(
    model="gemini-2.5-flash",  # or gemini-2.5-pro, gemini-2.0-flash, etc.
    api_base=os.getenv("LITELLM_PROXY_API_BASE"),
    api_key=os.getenv("LITELLM_PROXY_GEMINI_API_KEY"),
)

# ---- Create ADK agent ----
root_agent = Agent(
    name="ADK_WI_LiteLLM_agent",
    model=lite_llm_model,
    description="A simple agent using LiteLLM Virtual Keys for Gemini models",
    instruction="Always respond in English only and in maximum 3-bullet points."
)

root_agent

LlmAgent(name='ADK_WI_LiteLLM_agent', description='A simple agent using LiteLLM Virtual Keys for Gemini models', parent_agent=None, sub_agents=[], before_agent_callback=None, after_agent_callback=None, model=LiteLlm(model='gemini-2.5-flash', llm_client=<google.adk.models.lite_llm.LiteLLMClient object at 0x11903b360>), instruction='Always respond in English only and in maximum 3-bullet points.', global_instruction='', static_instruction=None, tools=[], generate_content_config=None, disallow_transfer_to_parent=False, disallow_transfer_to_peers=False, include_contents='default', input_schema=None, output_schema=None, output_key=None, planner=None, code_executor=None, before_model_callback=None, after_model_callback=None, on_model_error_callback=None, before_tool_callback=None, after_tool_callback=None, on_tool_error_callback=None)

In [11]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Create in-memory session service
session_service = InMemorySessionService()
app_name = "LiteLLM_Test_App"

# Create runner
runner = Runner(
    app_name=app_name,
    agent=root_agent,
    session_service=session_service,
)

# Define user and session IDs
user_id = "test_user"
session_id = "test_session_001"

# Create a new session
# Using await because create_session is an async method
session = await session_service.create_session(
    user_id=user_id,
    session_id=session_id,
    app_name=app_name
)
print(f"Created session: {session_id}")

# Create message content
new_message = types.Content(
    role="user",
    parts=[types.Part(text="Explain LiteLLM in one sentence.")]
)

# Run the agent and collect events
print("\nRunning agent...")
for event in runner.run(
    user_id=user_id,
    session_id=session_id,
    new_message=new_message
):
    if event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, 'text') and part.text:
                print(f"\n{event.author}: {part.text}")


Created session: test_session_001

Running agent...

ADK_WI_LiteLLM_agent: *   LiteLLM is an open-source proxy that simplifies and standardizes the use of various large language models (LLMs) from different providers, offering features like cost management, retries, and fallbacks.
